# Solving the OCP via indirect approach, forced Lagrangian multi-pendulum on a cart

first order eqs:
$$
\dot{x} = v_x, \dot{y}
$$


In [1]:
import sympy
import numpy as np
import function_definitions_forced as field_funcs
import helper_funcs as hfct
import pickle
import scipy
import scipy.optimize as opt
import OCP_construction_functions_forced as OCP_funcs
import ipdb


In [2]:
N_choice = 200
alpha_choice = 1
gamma_choice= 0.5
save_data_file = 'data/multi_pendulum_inversion/reference/two_pendulum_reference.pkl'

OCP_parameters = {
            # Generic OCP params, always needed
            "T": 3.,   #s
            # "dim_q": 2,  # state dimension will be set by the control Lagrangian
            # "dim_u": 1,  # actuation dimension will be set by the control Lagrangian
            "N":N_choice,     #240, 160, 80, 40, 20
            "q0": sympy.Matrix([0.,0,0.]),        #(m,1,1)
            "dq0": sympy.Matrix([0.,0.,0.]),       #(m/s,1/s,1/s)
            "qT": sympy.Matrix([0.,sympy.pi,0.]),      
            "dqT": sympy.Matrix([0.,0.,0.]),       #(vr,vphi1,vphi2)
            "alpha": alpha_choice,
            "gamma": gamma_choice,
            'folder_name': "multi_pendulum_inversion",
            #specific to model
            "m0":10.,
            "m1":9.,  # kg
            "m2":9.,
            "l1":0.5,
            "l2":0.5,
            "I1":3.2,
            "I2":3.2,
            "G": 9.81, #m/s**2
            "Aq": 123.7, #  kg/s^2
            "Adq": 127.7, # kg/s
            "A_u": 0.00052,
            "dim_u":3,
            "variable_names": ["x", "theta_1","theta_2"]
        }
OCP_parameters["h"] = sympy.symbols("h")
OCP_parameters["times"] = sympy.Matrix([i*OCP_parameters["h"] for i in range(OCP_parameters["N"]+1)])

# Function definition

In [3]:
continuous_eq = OCP_funcs.Direct_continuous_generator_forced_L_double_pendulum(OCP_parameters,Lagrangian=field_funcs.two_pendulum_cart_lagrangian
                                                               ,f_L=field_funcs.f_L_two_pendulum
                                                               ,running_cost_func=field_funcs.running_cost_two_pendulum
                                                               ,mayer_func=field_funcs.mayer_term_two_pendulum
                                                               ,g_func=field_funcs.g_mat_two_pendulum
                                                               ,I_func=field_funcs.conserved_I_control_two_pendulum)

# initial guess creation

In [4]:
OCP_parameters_np_version = hfct.sympy_to_np_dict(OCP_parameters,alpha_choice,gamma_choice)
parameters = OCP_parameters_np_version


with open(save_data_file, 'rb') as files:
    initial_guess_data = pickle.load(files)

q_d_start_guess = initial_guess_data["q_d_new"]
lam_d_start_guess = initial_guess_data["lambda_d_new"]
u_d_start_guess = initial_guess_data['u_d_new']

cs_q =  scipy.interpolate.CubicSpline(np.linspace(0,parameters["T"],len(q_d_start_guess)), q_d_start_guess.reshape([len(q_d_start_guess),3]))
cs_lam = scipy.interpolate.CubicSpline(np.linspace(0,parameters["T"],len(lam_d_start_guess)), lam_d_start_guess.reshape([len(lam_d_start_guess),3]))  
cs_u= scipy.interpolate.CubicSpline(np.linspace(0,parameters["T"],len(u_d_start_guess)),u_d_start_guess)

U_d_1_use = cs_u(np.array(parameters["times"]) + parameters["gamma"]*parameters["h"]).reshape(len(parameters["times"]),1,3)
U_d_2_use = cs_u(np.array(parameters["times"]) + (1-parameters["gamma"])*parameters["h"]).reshape(len(parameters["times"]),1,3)

q_d_use = cs_q(parameters["times"]).reshape(len(parameters["times"]),3,1)
lambda_d_use = cs_lam(parameters["times"]).reshape(len(parameters["times"]),3,1)
mu_use = initial_guess_data["mu_sol"] 
nu_use = initial_guess_data["nu_sol"] 
v_q_use =[]

import matplotlib.pyplot as plt
q_d_use = list(q_d_use)
lambda_d_use= list(lambda_d_use)
U_d_1_use = list(U_d_1_use)
U_d_2_use = list(U_d_2_use)
q_d_use = np.array(q_d_use)
lambda_d_use= np.array(lambda_d_use)
U_d_1_use = np.array(U_d_1_use)
U_d_2_use = np.array(U_d_2_use)



# Evolution via state eq

In [5]:
discrete_equations = OCP_funcs.discrete_standard_direct_eq_generator_forced_L_double_pendulum(continuous_eq)

# lambdify the equations for root finding

In [ ]:
#KKT direct approach with new Lagrangian
standard_direct_KKT_new = discrete_equations.calc_KKT_new(OCP_parameters_np_version)

lambdified_KKT_new =  sympy.lambdify(standard_direct_KKT_new[1],standard_direct_KKT_new[0].subs(discrete_equations.h,OCP_parameters_np_version["h"]).subs(discrete_equations.cont_equations.parameters["alpha"],alpha_choice).subs(discrete_equations.cont_equations.parameters["gamma"],gamma_choice))

lambdified_KKT_new_eval = lambda x :lambdified_KKT_new(*x)


# Executing root for solving KKT

In [ ]:
initial_guess_new = list(np.concatenate([mu_use,nu_use])) #mu and nu

for tmp in q_d_use:
    initial_guess_new+=list(tmp.flatten())
    #vq guess
#lambda terms
for tmp in lambda_d_use:
    initial_guess_new += list(tmp.flatten())


initial_guess_new += list(U_d_1_use.flatten()) 
initial_guess_new += list(U_d_2_use.flatten()) 


In [ ]:
#new lagrangian approach
standard_result_polar_KKT_new = opt.root(lambdified_KKT_new_eval,x0=initial_guess_new,method="lm")
standard_result_polar_KKT_new = opt.root(lambdified_KKT_new_eval,x0=standard_result_polar_KKT_new.x,method="lm")


In [ ]:
standard_result_polar_KKT_new = opt.root(lambdified_KKT_new_eval,x0=standard_result_polar_KKT_new.x)


In [ ]:
standard_result_polar_KKT_new

# Post-processing

In [ ]:
reshape_params = discrete_equations.cont_equations.parameters
reshape_N = reshape_params["N"]
reshape_dim_q = reshape_params["dim_q"]
reshape_dim_u = reshape_params["dim_u"]
mu_KKT_new,nu_KKT_new = standard_result_polar_KKT_new.x[:2*reshape_dim_q].reshape(2,reshape_dim_q)
q_d_KKT_new = standard_result_polar_KKT_new.x[2*reshape_dim_q:2*reshape_dim_q + reshape_dim_q * (reshape_N+1)].reshape(reshape_N + 1,reshape_dim_q)
lam_d_KKT_new = standard_result_polar_KKT_new.x[2*reshape_dim_q + reshape_dim_q * (reshape_N+1):2*reshape_dim_q + 2*reshape_dim_q * (reshape_N+1)].reshape(reshape_N + 1,reshape_dim_q)

U1_d_KKT_new = standard_result_polar_KKT_new.x[2*reshape_dim_q + 2*reshape_dim_q * (reshape_N+1):2*reshape_dim_q + 2*reshape_dim_q * (reshape_N+1) + (reshape_N+1)*reshape_dim_u].reshape(reshape_N + 1,reshape_params["dim_u"])
U2_d_KKT_new = standard_result_polar_KKT_new.x[2*reshape_dim_q + 2*reshape_dim_q * (reshape_N+1) + (reshape_N+1)*reshape_dim_u:].reshape(reshape_N + 1,reshape_params["dim_u"])

p_y_d_new = np.array([sympy.flatten(tmp) for tmp in discrete_equations.p_v_d_from_y_d(q_d_KKT_new,lam_d_KKT_new,U1_d_KKT_new,U2_d_KKT_new,OCP_parameters_np_version)])

v_y_d_new = []
for tmp1,tmp2,tmp3 in zip(q_d_KKT_new,lam_d_KKT_new,p_y_d_new):
    v_y_d_new.append(sympy.flatten(discrete_equations.cont_equations.calc_vy_fromq_p_new(tmp1,tmp2,tmp3,OCP_parameters_np_version)))
v_y_d_new = np.array(v_y_d_new) 


u_vec_KKT_new = []
for i in range(len(q_d_KKT_new)):
    u_vec_KKT_new.append(discrete_equations.cont_equations.u_expr_from_new_lambdified(*q_d_KKT_new[i],*lam_d_KKT_new[i],*v_y_d_new[i]).flatten())

u_vec_KKT_new = np.array(u_vec_KKT_new)

H_control_KKT_new = []
for i in range(len(q_d_KKT_new)):
    H_control_KKT_new.append(discrete_equations.cont_equations.new_control_H_v_calc(q_d_KKT_new[i],lam_d_KKT_new[i],v_y_d_new[i],u_vec_KKT_new[i],OCP_parameters_np_version))
H_control_KKT_new = np.array(H_control_KKT_new)  

I_d_evo_new = np.array(p_y_d_new.transpose()[0],float)

# plot results

In [ ]:
q_d_new_KKT_sol_cart = []

q_d_new_KKT_sol_cart = q_d_KKT_new    
q_d_new_KKT_sol_cart = np.array(q_d_new_KKT_sol_cart)


plt.plot(OCP_parameters_np_version["times"],q_d_new_KKT_sol_cart.transpose()[0],'--',label='new x evo')
plt.plot(OCP_parameters_np_version["times"],q_d_new_KKT_sol_cart.transpose()[1],'--',label='new theta1 evo')
plt.plot(OCP_parameters_np_version["times"],q_d_new_KKT_sol_cart.transpose()[2],'--',label='new theta2 evo')
plt.legend()
plt.xlabel('t')
plt.ylabel('y')
plt.tight_layout()

plt.show()

plt.plot(OCP_parameters_np_version["times"],lam_d_KKT_new.transpose()[0],'--',label='new approach lam_r')
plt.legend()
plt.xlabel('t')
plt.ylabel('lambda_r')
plt.tight_layout()


plt.show()
plt.plot(OCP_parameters_np_version["times"],lam_d_KKT_new.transpose()[1],'--',label='new aproach lam_phi')
plt.legend()
plt.xlabel('t')
plt.ylabel('lambda_phi')
plt.tight_layout()



plt.show()
plt.plot(U1_d_KKT_new[:-1].transpose()[0],'x',label='new u0')
plt.plot(U1_d_KKT_new[:-1].transpose()[1],'x',label='new u1')
plt.plot(U1_d_KKT_new[:-1].transpose()[2],'x',label='new u2')
plt.plot(U_d_1_use[:-1,0,0],'--',label='comparison1')
plt.plot(U_d_1_use[:-1,0,1],'--',label='comparison2')
plt.plot(U_d_1_use[:-1,0,2],'--',label='comparison3')
plt.legend()
plt.xlabel('N step')
plt.ylabel('U_1')
plt.tight_layout()
plt.show()

plt.plot(U2_d_KKT_new[:-1],'x',label='new')
plt.plot(U_d_2_use[:-1,0,0],'--',label='comparison1')
plt.plot(U_d_2_use[:-1,0,1],'--',label='comparison2')
plt.plot(U_d_2_use[:-1,0,2],'--',label='comparison3')
plt.legend()
plt.xlabel('N step')
plt.ylabel('U_2')
plt.tight_layout()


plt.show()
plt.plot(U_d_2_use[:-1,0,0],'--',label='comparison')
plt.plot(u_vec_KKT_new[:,0],label='new u0d')
plt.plot(u_vec_KKT_new[:,1],label='new u1d')
plt.plot(u_vec_KKT_new[:,2],label='new u2d')
plt.xlabel('N step')
plt.ylabel('u_d')
plt.legend()

plt.show()
plt.plot((np.array(I_d_evo_new)-np.array(I_d_evo_new)[0]),'X-',label="I new")
plt.legend()
plt.xlabel('N step')
plt.ylabel('I')
plt.tight_layout()

plt.show()
plt.plot((H_control_KKT_new[:-1]),'-*',label='new control H')
plt.legend()


# Saving data

In [ ]:
from pathlib import Path 

if not standard_result_polar_KKT_new.success:
    print('did not find a solution in control dependent case, not storing the result')
else:
    storage_dict = dict()
    storage_dict["parameters"] = OCP_parameters_np_version
    storage_dict["q_d_new"] = q_d_KKT_new
    storage_dict["q_d_new_cartesian"] = q_d_new_KKT_sol_cart
    storage_dict["lambda_d_new"] = lam_d_KKT_new
    storage_dict["p_y_d_new"] = np.array(p_y_d_new,dtype=float)
    storage_dict["U1_d_new"] = np.array(U1_d_KKT_new,dtype=float)
    storage_dict["U2_d_new"] = np.array(U2_d_KKT_new,dtype=float)
    storage_dict["u_d_new"] = np.array(u_vec_KKT_new,dtype=float)
    storage_dict["I_d_new"] = np.array(I_d_evo_new,dtype=float)
    storage_dict["H_control_new"] = np.array(H_control_KKT_new,dtype=float)
    storage_dict["v_y_d_new"] = np.array(v_y_d_new,dtype=float)
    storage_dict["mu_sol"] = np.array(mu_KKT_new,dtype=float)
    storage_dict["nu_sol"] = np.array(nu_KKT_new,dtype=float)


    dirpath = "data/" + OCP_parameters_np_version["folder_name"]+"/data_"+"a=" + str(OCP_parameters_np_version["alpha"])  +"g=" + str(OCP_parameters_np_version["gamma"])
    Path(dirpath).mkdir(parents=True, exist_ok=True)

    file_name = dirpath+"/data_" +"a=" + str(OCP_parameters_np_version["alpha"])  +"g=" + str(OCP_parameters_np_version["gamma"])  +"N=" + str(OCP_parameters_np_version["N"]) + ".pkl"
    with open(file_name, 'wb') as ffile:
        pickle.dump(storage_dict, ffile)
        ffile.close()